<a href="https://colab.research.google.com/github/saivenkatreddy29/Implementation-of-Hands-on-Large-Language-models-book/blob/main/Jaylamar_book_chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-3-mini-4k-instruct",
    device_map = 'cuda',
    torch_dtype = 'auto',
    trust_remote_code = True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-3-mini-4k-instruct')

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    return_full_text = False,
    max_new_tokens = 500,
    do_sample = 500
)

In [ ]:
messages = [
    {"role": "user", "content": "create a funny joke about chickens"}
]

output = pipe(messages)
print(output[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Why don't chickens use Facebook? Because they can't stop sharing pictures of eggs and then someone always comes along and "lays" a fresh joke!


In [ ]:
prompt = pipe.tokenizer.apply_chat_template(messages,tokenize = False)
print(prompt)

<|user|>
create a funny joke about chickens<|end|>
<|endoftext|>


In [ ]:
output = pipe(messages, do_sample = True, temperature = 0.0001)
print(output)

[{'generated_text': ' Why did the chicken join the band? Because it had the drumsticks!'}]


In [ ]:
output = pipe(messages, do_sample = True, top_p = 0.01)
print(output)

[{'generated_text': ' Why did the chicken join the band? Because it had the drumsticks!'}]


# **Zero Shot Chain of thought**

In [ ]:
zeroshot_cot_prompt = [
    {"role": "user", "content": "The cafeteria has 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? Let's think step by step."}

]

# Generate the output
outputs = pipe(zeroshot_cot_prompt)

In [ ]:
print(outputs[0]['generated_text'])

 To find out how many apples the cafeteria has now, we subtract the apples used and then add the apples they bought:

1. Start with the original number of apples: 23 apples
2. Subtract the number of apples used to make lunch: 23 apples - 20 apples = 3 apples
3. Add the number of apples bought: 3 apples + 6 apples = 9 apples

So, the cafeteria now has 9 apples.


In [ ]:
zeroshot_cot_prompt = [
    {"role": "user", "content": "The cafeteria has 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"}

]

# Generate the output
outputs = pipe(zeroshot_cot_prompt)
print(outputs[0]['generated_text'])

In [ ]:
zeroshot_tot_prompt = [
    {"role": "user","content": "Imageing three different experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realises they're wrong at any point then they leave. Teh question is 'The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?' Make sure to discuss the results "}
]

In [ ]:
outputs = pipe(zeroshot_tot_prompt)
print(outputs[0]['generated_text'])

 Expert 1 thought process:

1. I'll subtract the apples used to make lunch from the initial count: 23 - 20 = 3 apples left.

Expert 2 thought process:

1. Determine the initial number of apples: 23.
2. Subtract the apples used for lunch: 23 - 20 = 3 apples left.
3. Add the new apples purchased: 3 + 6 = 9 apples.

Expert 3 thought process:

1. Start with the initial amount of 23 apples.
2. To find out the remaining apples after using some, subtract 20 from 23: 23 - 20 = 3 apples remaining.
3. Next, to calculate the final count, I'll add the 6 new apples purchased: 3 + 6 = 9 apples.

Consensus:

All experts have come to the same conclusion, which is that the cafeteria now has 9 apples left after making lunch and purchasing more.


# **Grammer Constrained Sampling**

In [ ]:
import gc
import torch
del model, tokenizer, pipe

gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3510596 sha256=b53daa5345dda34ab9ad260e0bd486ac757a4f0b83f45c258e162829fbee45d8
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [ ]:
from llama_cpp.llama import Llama

# #load phi-3
llm = Llama.from_pretrained(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="*fp16.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    verbose=False
)

Phi-3-mini-4k-instruct-fp16.gguf:   0%|          | 0.00/7.64G [00:00<?, ?B/s]